# Flatmap data

This notebook creates plots of the polar and eccentricity data that was projected to the flatmap by the SUIT toolbox.

In [4]:
import numpy as np
import pandas as pd
import pycircstat as pc
import copy as copy
import os
import nibabel as nb

import matplotlib.pyplot as pl
import seaborn as sn
sn.set_style('ticks')
import glob

import matplotlib as mpl
%matplotlib inline
mpl.rc_file_defaults()

In [8]:
# define directories
base_dir = '/home/shared/2018/visual/hcp_cerebellum/'
repo_dir  = '/home/vanes/git/hcp_cerebellum/'
suit_home  = '/home/vanes/bin/suit/'

resource_dir = os.path.join(repo_dir,'resources')
flat_dir = os.path.join(repo_dir,'flat_data')



# setup figure directory
fig_dir = os.path.join(base_dir,'figs')
if not os.path.isdir(fig_dir): os.mkdir(fig_dir)
    

In [9]:
# these are the coordinates of the SUIT flat data:
# in the toolbox, these are located in $spmhome/suit/flatmap
coords=nb.load(os.path.join(suit_home,'flatmap','FLAT.coord.gii'))

x = np.ravel(coords.get_arrays_from_intent(1008)[0].data[:,0]) 
y = np.ravel(coords.get_arrays_from_intent(1008)[0].data[:,1]) 

In [10]:
# get the flatmap representations of the retmaps
masks=np.ravel(pd.read_csv(os.path.join(flat_dir,'retmaps_flat.csv'),header=None))

In [11]:
roi_order = ['left_OMV','right_OMV','left_VIIb','right_VIIb','left_VIIIb','right_VIIIb']

these_combs = {
    'left_OMV':['left_mOMV','left_lOMV'],
    'right_OMV':['right_mOMV','right_lOMV'],
    'left_VIIb':['left_VIIb'],
    'right_VIIb':['right_VIIb'],
    'left_VIIIb':['left_mVIIIb','left_lVIIIb'],
    'right_VIIIb':['right_mVIIIb','right_lVIIIb'],    
}

In [12]:
def rotate_via_numpy(x,y, radians):
    """Use numpy to build a rotation matrix and take the dot product."""
    c, s = np.cos(radians), np.sin(radians)
    j = np.matrix([[c, s], [-s, c]])
    m = np.dot(j, np.matrix([x, y]))

    return m

In [13]:
def get_roi_color(mask):
    
    # determine base roi colors
    if mask in ['left_OMV','right_OMV','OMV']:
        c = '#6590CB'
    elif mask in ['left_VIIIb','right_VIIIb','VIIIb']:
        c = '#E55D5C'
    elif ('VIIb' in mask) + (mask=='VIIb'):
        c = '#E79F2A'
        
    # roi colors for lateral / medial separation:
    elif mask in ['left_lOMV','right_lOMV','lOMV']:
        c = '#6590CB'
    elif mask in ['left_mOMV','right_mOMV','mOMV']:
        c = '#0D5B99'
    elif mask in ['left_mVIIIb','right_mVIIIb','mVIIIb']:
        c = '#C03142'
    elif mask in ['left_lVIIIb','right_lVIIIb','lVIIIb']:
        c = '#E57F80'
        
    return c

In [17]:
# these refer to the indices in cerebellum_retmaps.nii
mask_names ={
'left_mOMV':7,
'right_mOMV':9,
'left_lOMV':8,
'right_lOMV':10,
    
'left_VIIb':5,
'right_VIIb':6,
    
'left_mVIIIb':3,
'right_mVIIIb':2,
'left_lVIIIb':4,
'right_lVIIIb':1,
}

# this is how the lateral/medial and left right hemisphere indices combine:
roi_combs = {
    'left_OMV':[7,8],
    'right_OMV':[9,10],
    'left_VIIb':[5],
    'right_VIIb':[6],
    'left_VIIIb':[3,4],
    'right_VIIIb':[1,2],

    'OMV':[7,8,9,10],
    'lOMV':[8,10],
    'mOMV':[7,9],
    'VIIb':[5,6],
    'VIIIb':[1,2,3,4],
    'lVIIIb':[1,4],
    'mVIIIb':[2,3],
}

roi_order = ['left_OMV','right_OMV','left_VIIb','right_VIIb','left_VIIIb','right_VIIIb']
# roi_comb_order = ['mOMV','lOMV','VIIb','mVIIIb','lVIIIb']           

In [15]:
kernel = 6 # 1/kernel is the proportion of data in single moving average operation

### polar maps

In [19]:
pl.close('all')
angles=np.ravel(pd.read_csv(os.path.join(flat_dir,'ang_flatdata_avgsj.csv'),header=None))
r2=np.ravel(pd.read_csv(os.path.join(flat_dir,'r2_flatdata_avgsj.csv'),header=None))

# rotate angles so 0 is up
angles += 1.5*np.pi
angles = np.mod(angles,np.pi*2)

# angles
rot_mask = {
    'left_OMV':0.75,
    'right_OMV':0.25,
    'left_VIIb':1.25,
    'right_VIIb':1.75,
    'left_VIIIb':0.75,
    'right_VIIIb':0.25,

}    

f = pl.figure(figsize=(1.7,2.75))

for mi,mask in enumerate(roi_order):
    
    c = get_roi_color(mask)

    s = f.add_subplot(3,2,mi+1)

    # create mask
    v = np.zeros_like(masks).astype(bool)
    for subroi in these_combs[mask]:        
        v += (masks==mask_names[subroi])
    v *= (np.invert(np.isnan(angles)))
     
    # rotate x values
    rotation = rot_mask[mask]
    rx = np.ravel(rotate_via_numpy(x,y,rotation*np.pi)[0])

    # invert angle data if left hemisphere
    if 'left' in mask:
        angledata = copy.copy(angles)
        angledata = 2*np.pi-angledata
    elif 'right' in mask:
        angledata = copy.copy(angles)

    # create moving average
    # first sort values based on x
    order = np.argsort(rx[v])
    xo = rx[v][order]
    yo = angledata[v][order]
    wo = r2[v][order]
    n = v.sum() 
    kw = int(n/kernel)
    
    # set nan weights to 0
    wo[np.isnan(wo)] = 0
    
    ma_x = np.array([np.average(xo[i:i+kw],weights=wo[i:i+kw]) for i in range(n-kw)]) # these are the x coords
    ma_y = np.array([pc.mean(yo[i:i+kw],w=wo[i:i+kw]) for i in range(n-kw)]) # these are the angles
    
    ma_y = np.degrees(ma_y)

    pl.plot(ma_x,ma_y,lw=1.5,color=c)

    if 'VIIb' not in mask:
        pl.axvline(ma_x[np.argmin(ma_y)],ls='--',c='k',dashes=(2,2))
        
    if 'left' in mask:
        pl.ylim(160,290)
        pl.yticks([180,270],['down','left'])
    elif 'right' in mask:
        pl.ylim(160,290)
        pl.yticks([180,270],['down','right'])
    
    pl.xticks([np.min(ma_x),np.max(ma_x)],['0','%.1f'%(np.max(ma_x)-np.min(ma_x))])

    sn.despine(offset=2)

pl.tight_layout()
f.savefig(os.path.join(fig_dir,'anglesurf_cluster.pdf'))

### now do the same for eccentricity

In [27]:
eccs=np.ravel(pd.read_csv(os.path.join(flat_dir,'ecc_flatdata_avgsj.csv'),header=None))
r2=np.ravel(pd.read_csv(os.path.join(flat_dir,'r2_flatdata_avgsj.csv'),header=None))

# angles
rot_mask = {
    'left_OMV':0.25,
    'right_OMV':0.75,
    'left_VIIb':0,
    'right_VIIb':1,
    'left_VIIIb':0.25,
    'right_VIIIb':0.75,
}

f = pl.figure(figsize=(1.5,2.75))

for mi,mask in enumerate(roi_order):
    
    c = get_roi_color(mask)

    s = f.add_subplot(3,2,mi+1)

    # create mask
    v = np.zeros_like(masks).astype(bool)
    for subroi in these_combs[mask]:        
        v += (masks==mask_names[subroi])
    v *= (np.invert(np.isnan(eccs)))
     
    # rotate x values
    rotation = rot_mask[mask]
    rx = np.ravel(rotate_via_numpy(x,y,rotation*np.pi)[0])

    data = copy.copy(eccs)

    order = np.argsort(rx[v])
    xo = rx[v][order]
    yo = data[v][order]
    wo = r2[v][order]
    n = v.sum() 
    kw = int(n/kernel)

    # set nan weights to 0
    wo[np.isnan(wo)] = 0    

    ma_x = np.array([np.average(xo[i:i+kw],weights=wo[i:i+kw]) for i in range(n-kw)]) # these are the x coords
    ma_y = np.array([pc.mean(yo[i:i+kw],w=wo[i:i+kw]) for i in range(n-kw)]) # these are the angles

    pl.plot(ma_x,ma_y,lw=1.5,color=c)

    if (mask == 'left_OMV') + (mask == 'right_OMV'):
        pl.yticks([0,0.5],['0','0.5'])
        pl.ylim(0,0.5)
    elif (mask == 'left_VIIb') + (mask == 'right_VIIb'):
        pl.yticks([0,4],['0','4.0'])
        pl.ylim(0,4)
    elif (mask == 'left_VIIIb') + (mask == 'right_VIIIb'):
        pl.yticks([0,8],['0','8.0'])
        pl.ylim(0,8)

    pl.xticks([np.min(ma_x),np.max(ma_x)],['0','%.1f'%(np.max(ma_x)-np.min(ma_x))])

    sn.despine(offset=2)

pl.tight_layout()
f.savefig(os.path.join(fig_dir,'eccsurf_cluster.pdf'))#,dpi=500)